### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = English
Lang Id = 1
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,and this is what it was for me and it was,2038,2039,and this is what it was for me and it was great,mO0oqUQ5eHc,https://www.youtube.com/watch?v=mO0oqUQ5eHc&t=...
1,what do we have to do and not do it this,2007,2008,but ok we dont what do we have to do and not d...,7eUrrQRpz2w,https://www.youtube.com/watch?v=7eUrrQRpz2w&t=...
2,and what this will do is this will,2226,2227,and what this will do is this will,UNnFRj9_TNo,https://www.youtube.com/watch?v=UNnFRj9_TNo&t=...
3,and what this will do is this will,274,275,and what this will do is this will,fgMD2wvpvpk,https://www.youtube.com/watch?v=fgMD2wvpvpk&t=...
4,what it is that you do and do not,624,625,what it is that you do and do not like,Gyqu5AER7gg,https://www.youtube.com/watch?v=Gyqu5AER7gg&t=...
...,...,...,...,...,...,...
195,that was it and in,466,467,so that was it and in,1keqwqfBtp4,https://www.youtube.com/watch?v=1keqwqfBtp4&t=...
196,it was not in the,6980,6981,it was not in the tfl,0WmwRiX0sxo,https://www.youtube.com/watch?v=0WmwRiX0sxo&t=...
197,for me it was the,138,139,for me it was the,1up0ZaqKSAA,https://www.youtube.com/watch?v=1up0ZaqKSAA&t=...
198,i was not in it i,1296,1297,i was not in it i did,Yamhz_I4TRY,https://www.youtube.com/watch?v=Yamhz_I4TRY&t=...


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,so do you think life will work in the right wa...,560,561,so do you think life will work in the right wa...,IXhNNNeYXnA,https://www.youtube.com/watch?v=IXhNNNeYXnA&t=...
1,what do you think about this i mean yeah to me...,538,539,what do you think about this i mean yeah to me...,_aBzOI9DWb4,https://www.youtube.com/watch?v=_aBzOI9DWb4&t=...
2,things that are out there you can do nothing a...,937,938,things that are out there you can do nothing a...,ldlflaQVAAE,https://www.youtube.com/watch?v=ldlflaQVAAE&t=...
3,the way is where this is why i really want to ...,2610,2611,the way is where this is why i really want to ...,MXGmKbjEIlw,https://www.youtube.com/watch?v=MXGmKbjEIlw&t=...
4,say oh yeah yes she can and she would like to ...,1637,1638,say oh yeah yes she can and she would like to ...,dYa5scWadpc,https://www.youtube.com/watch?v=dYa5scWadpc&t=...
...,...,...,...,...,...,...
195,the other people but it can be about us right ...,658,659,the other people but it can be about us right ...,u5tDsCrIacA,https://www.youtube.com/watch?v=u5tDsCrIacA&t=...
196,as long as more people are doing better than n...,334,335,as long as more people are doing better than n...,ZiMBNuSObjo,https://www.youtube.com/watch?v=ZiMBNuSObjo&t=...
197,are we know nothing about them they go right t...,791,792,the recent recruits are we know nothing about ...,4yRKUDQ8PLE,https://www.youtube.com/watch?v=4yRKUDQ8PLE&t=...
198,a very long time of something like this it wou...,3100,3101,a very long time of something like this it wou...,SmS3fXDLruQ,https://www.youtube.com/watch?v=SmS3fXDLruQ&t=...


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,200,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,so you can imagine if you go through all of th...,293,294,so you can imagine if you go through all of th...,K1GhQ8633Bg,https://www.youtube.com/watch?v=K1GhQ8633Bg&t=...
2,and so one of the questions people say to me i...,971,972,craig robinson and so one of the questions peo...,G3w3f6M_w54,https://www.youtube.com/watch?v=G3w3f6M_w54&t=...
3,its a little family that you turn into and jus...,49,50,its a little family that you turn into and jus...,GymR27z8RHM,https://www.youtube.com/watch?v=GymR27z8RHM&t=49s
4,and i remember when it happened i was like oh ...,796,797,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0,https://www.youtube.com/watch?v=M_IRnJZ2xQ0&t=...
...,...,...,...,...,...,...
195,yeah maybe your dog will be okay for a while u...,1448,1449,yeah maybe your dog will be okay for a while u...,3Q_FPPslCWs,https://www.youtube.com/watch?v=3Q_FPPslCWs&t=...
196,this might not be for quite some time it could...,456,457,this might not be for quite some time it could...,jC7rfXRpNp0,https://www.youtube.com/watch?v=jC7rfXRpNp0&t=...
197,months of age we did not leave any child of co...,1608,1609,months of age we did not leave any child of co...,42jHLEhgKwA,https://www.youtube.com/watch?v=42jHLEhgKwA&t=...
198,not sure what these other guys are talking abo...,2822,2823,im not sure what these other guys are talking ...,fYVuYbCvsiQ,https://www.youtube.com/watch?v=fYVuYbCvsiQ&t=...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,without having um as much structure or as much...,2093,2094,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0,https://www.youtube.com/watch?v=iJkfTPN8MS0&t=...
1,lets talk about your action plans now and i an...,1870,1871,lets talk about your action plans now and i an...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
2,here in as an example of hours of operation ri...,933,934,here in as an example of hours of operation ri...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
3,norman jones remember the kind of big picture ...,2320,2321,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
4,de rose we appreciate you being here im glad i...,2799,2800,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
...,...,...,...,...,...,...
195,boxes were but this one will actually come rig...,490,491,boxes were but this one will actually come rig...,cXy2u6Utcxo,https://www.youtube.com/watch?v=cXy2u6Utcxo&t=...
196,what happens it says now it technically report...,1214,1215,what happens it says now it technically report...,nHVh1Rqb9YY,https://www.youtube.com/watch?v=nHVh1Rqb9YY&t=...
197,on april first many western countries celebrat...,733,734,on april first many western countries celebrat...,xhJWVbZcNvw,https://www.youtube.com/watch?v=xhJWVbZcNvw&t=...
198,states plan in fact some states have more than...,3334,3335,states plan in fact some states have more than...,Pi1e9M8f5ME,https://www.youtube.com/watch?v=Pi1e9M8f5ME&t=...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,theresa anderson and because i only talked wit...,2089,2090,theresa anderson and because i only talked wit...,iZcte874ICY,https://www.youtube.com/watch?v=iZcte874ICY&t=...
1,dr michelle taylor good afternoon everyone its...,2165,2166,dr michelle taylor good afternoon everyone its...,rZEc_NCcorU,https://www.youtube.com/watch?v=rZEc_NCcorU&t=...
2,norman jones remember the kind of big picture ...,2320,2321,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
3,de rose we appreciate you being here im glad i...,2799,2800,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
4,when all the students parents old school toget...,3051,3052,portia polk when all the students parents old ...,6dxtMxcFJ3M,https://www.youtube.com/watch?v=6dxtMxcFJ3M&t=...
...,...,...,...,...,...,...
195,at cloud privacy labs we help machines underst...,1145,1146,at cloud privacy labs we help machines underst...,wWmAOiJGWI0,https://www.youtube.com/watch?v=wWmAOiJGWI0&t=...
196,so finn is an emotional support animal so he h...,514,515,so finn is an emotional support animal so he h...,1rcBcbM9G8g,https://www.youtube.com/watch?v=1rcBcbM9G8g&t=...
197,yes in soccer whats important is not winning o...,2,3,yes in soccer whats important is not winning o...,MaA29LZLAHQ,https://www.youtube.com/watch?v=MaA29LZLAHQ&t=2s
198,even though there are several tools available ...,45,46,even though there are several tools available ...,u3WECstfzRE,https://www.youtube.com/watch?v=u3WECstfzRE&t=45s


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,that this is not just getting any old attorney...,5513,5514,katie kersh that this is not just getting any ...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,thats something you dont have to worry about a...,2199,2200,lauren opgenorth thats something you dont have...,HWTd4jxCixk,https://www.youtube.com/watch?v=HWTd4jxCixk&t=...
2,so prepare relevant questions and also on the ...,1444,1445,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs,https://www.youtube.com/watch?v=qgBqBnEJFgs&t=...
3,james moore depending on the graduation someti...,3463,3464,james moore depending on the graduation someti...,9nBekn1eQtg,https://www.youtube.com/watch?v=9nBekn1eQtg&t=...
4,this type of research yeah i think thats still...,3706,3707,this type of research yeah i think thats still...,Anatl2PLRag,https://www.youtube.com/watch?v=Anatl2PLRag&t=...
...,...,...,...,...,...,...
195,so this is about how much dough we want per co...,232,233,so this is about how much dough we want per co...,Bfgl3ng5MM4,https://www.youtube.com/watch?v=Bfgl3ng5MM4&t=...
196,things are feeling kinda final girl right abou...,722,723,things are feeling kinda final girl right abou...,sT0gJbZDvL4,https://www.youtube.com/watch?v=sT0gJbZDvL4&t=...
197,or having your crew pilot separate fighters in...,470,471,or having your crew pilot separate fighters in...,AOSMF5G08tY,https://www.youtube.com/watch?v=AOSMF5G08tY&t=...
198,today i performed secondary surgery in another...,909,910,today i performed secondary surgery in another...,SeBK4nRZWek,https://www.youtube.com/watch?v=SeBK4nRZWek&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,brian hoffman but look at how different the ou...,1756,1757,brian hoffman but look at how different the ou...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,chrissy foster were seeing this pop up in so m...,5444,5445,chrissy foster were seeing this pop up in so m...,SnAFp1hTNP8,https://www.youtube.com/watch?v=SnAFp1hTNP8&t=...
2,and that is something thats really important e...,485,486,jillian deas and that is something thats reall...,twJkEtTvvEo,https://www.youtube.com/watch?v=twJkEtTvvEo&t=...
3,david shoemaker responded back to her in a pri...,2541,2542,david shoemaker responded back to her in a pri...,PRG60EnKnIo,https://www.youtube.com/watch?v=PRG60EnKnIo&t=...
4,teresa bill well and ill just share that i rea...,2082,2083,teresa bill well and ill just share that i rea...,tffd1kmM3e4,https://www.youtube.com/watch?v=tffd1kmM3e4&t=...
...,...,...,...,...,...,...
195,right now it would look quite different with n...,8,9,right now it would look quite different with n...,5zrP6VH88qU,https://www.youtube.com/watch?v=5zrP6VH88qU&t=8s
196,we came pretty far on this channel already so ...,591,592,we came pretty far on this channel already so ...,y7vjpxz_ymI,https://www.youtube.com/watch?v=y7vjpxz_ymI&t=...
197,i was gon na say what lame children wow this i...,103,104,i was gon na say what lame children wow this i...,h6UeoGWlYyU,https://www.youtube.com/watch?v=h6UeoGWlYyU&t=...
198,on there no classrooms open area high ceiling ...,339,340,on there no classrooms open area high ceiling ...,JzE_OILmCL8,https://www.youtube.com/watch?v=JzE_OILmCL8&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1674926273
  nanos: 133526000
}